In [1]:
#Importing librarires
import cv2
import numpy as np
import math

In [ ]:
#accessing laptop camera using opencv
cam = cv2.VideoCapture(0)
while(cam.isOpened()):
    #Reading an image
    ret, img = cam.read()
    
    #Collecting hand data from boxplot
    cv2.rectangle(img, (300,300), (100,100), (0,255,0),0)
    cropped  = img[100:300, 100:300]
    
    
    #Preprocessing the Image
    
    #Converting the captures image to grayscale
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    
    
    #Smoothing the image using Guassian Blur
    values = (35,35)
    blurred_img = cv2.GaussianBlur(gray, values, 0)
    
    
    #Applying threshold , Converting gray scale image to binary image using Otsu's method
    _, thresh1 = cv2.threshold(blurred_img, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    
    #Displaying the binary Image
    cv2.imshow('Binary Image', thresh1)
    
    
    (version, _, _) = cv2.__version__.split('.')
    
    
    if(version == '3'):
        image, contours, hierarchy = cv2.findContours(thresh1.copy(),  cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        
    elif(version == '4'):
        contours, hierarchy = cv2.findContours(thresh1.copy(),  cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        
        
        
    #Now finding with contour with max area
    
    max_count = max(contours, key = lambda x:cv2.contourArea(x))
    
    
    #Creating a bounding box aroud the contour
    x, y, w, h = cv2.boundingRect(max_count)
    cv2.rectangle(cropped, (x,y), (x+w, y+h), (0, 0, 255), 0)
    
    
    
    #finding the convex hull
    hull = cv2.convexHull(max_count)
    
    
    
    #drawing contour
    drawing_cont = np.zeros(cropped.shape, np.uint8)
    cv2.drawContours(drawing_cont, [max_count], 0, (0, 255, 0), 0)
    cv2.drawContours(drawing_cont, [hull], 0, (0, 0, 255), 0)
    
    
    #finding the convex hull
    hull = cv2.convexHull(max_count, returnPoints= False)
    
    
    
    #finding convexity defects
    defects = cv2.convexityDefects(max_count, hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0, 255, 0), 3)
    
    
    #applying cosine rule to fins all defectcs(angle between fingers)
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i,0]
        
        
        start = tuple(max_count[s][0])
        
        end = tuple(max_count[e][0])
        
        far = tuple(max_count[f][0])
        
        
        #finding length of all sides of triangles
        
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2 )
    
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2 )
        
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2 )
        
        
        #Now cosine rule
        
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c))* 57
        
        
        #ignoring if angle > 90 and highlight the rest with red dots
        
        if(angle <= 90):
            count_defects+=1
            cv2.circle(cropped, far, 1, [0,0,255], -1)
            
        
        
        #Draw a line from start to end(i.e the convex points(finger tips)
        
        cv2.line(cropped, start, end, [0,255,0], 2)
        
        
        
    if(count_defects == 1):
        cv2.putText(img, "Detecting a single Finger", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif(count_defects == 2):
        cv2.putText(img, "Detecting  Two Fingers", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif(count_defects == 3):
        cv2.putText(img, "Detecting Three Fingers", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    else:
        cv2.putText(img, "Detecting Four Fingers", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
            
            
            
    cv2.imshow('Gesture', img)
    all_img = np.hstack((drawing_cont, cropped))
    cv2.imshow('Contours', all_img)
    
    
    k = cv2.waitKey(10)
    if(k == 27):
        break
            
            
    
    
        